In [1]:
import chess
import chess.engine

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [2]:
from generate_data import *

In [3]:
class ChessDataset(Dataset):
    """Chess Positions Evaluation Dataset"""

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        position = np.array(self.df.iloc[idx, :-1])
        eval_ = np.array(self.df.iloc[idx, -1]/100)
        
        sample = {'position': torch.from_numpy(position), 'eval': torch.from_numpy(eval_)}

        return sample

In [4]:
dataset = ChessDataset(csv_file='data/stockfish_depth0e.csv')

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(8, 8, 2) #input, output channels, kernel_size
        self.conv2 = nn.Conv2d(8, 8, 3)
        self.conv3 = nn.Conv2d(8, 8, 3)
        self.conv4 = nn.Conv2d(8, 8, 3)
        self.fc1 = nn.Linear(8, 64) 
        self.fc2 = nn.Linear(64, 1) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)   
        x = torch.reshape(x, (-1,)) # flattens into 1d tensor
        x = self.fc1(F.relu(x))
        x = self.fc2(torch.sigmoid(x))
        #x = self.fc1(x)
        #x = self.fc2(x)

        return x

In [6]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)

net = Net()#.cuda()

In [7]:
t = torch.tensor([[1., -1.], [1., -1.]])
t = t.cuda()
t.get_device()

0

# TRAINING DATA

In [8]:
#torch.cuda.get_device_name(device)

In [9]:
trainloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0) 
#num_workers = 0 doesn't use GPU. GPU seems to be a lot slower than CPU,.
#data set is prob too big or my laptop nvidia GPU is just trash

optimizer = optim.Adam(net.parameters(), lr = 0.0009) #lr = 0.001
criterion = nn.MSELoss() 

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['position'].reshape(1, 8, 8, 8).float()#.cuda()
        labels = data['eval'].float()#.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 400 == 399:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 400))
            running_loss = 0.0

print('Finished Training')

RuntimeError: shape '[1, 8, 8, 8]' is invalid for input of size 513

In [10]:
def compare_eval():
    board = generate_rand_board()
    torch_board = torch.from_numpy(convert_board(board).reshape(1, 8, 8, 8)).float()
    net_eval = net(torch_board)
    sf_eval = stockfish_evaluation(board)
    print(f"NN: {net_eval} Stockfish: {sf_eval/100}")
    

In [11]:
for _ in range(5):
    compare_eval()

NN: tensor([0.5485], grad_fn=<AddBackward0>) Stockfish: -1.84
NN: tensor([8.6144], grad_fn=<AddBackward0>) Stockfish: 2.63
NN: tensor([-0.1055], grad_fn=<AddBackward0>) Stockfish: 1.0
NN: tensor([7.3383], grad_fn=<AddBackward0>) Stockfish: 2.37
NN: tensor([4.6139], grad_fn=<AddBackward0>) Stockfish: 1.94


In [12]:
# Choosing Best Move with Minimax

In [13]:
def nn_prediction(position: chess.Board):
    board_rep = convert_board(position)
    torch_board = torch.from_numpy(convert_board(board).reshape(1, 8, 8, 8)).float()
    return net(torch_board)


In [14]:
def find_best_move(position: chess.Board, depth: int = 0):
    best_evaluation = -1000
    
    for move in position.legal_moves:
        position.push(move)
        if position.turn == chess.WHITE:
            value = minimax(position, True, depth - 1) 
        else:
            value = minimax(position, False, depth - 1) 
        position.pop()
        if value > best_evaluation:
            best_evaluation = value
            best_move = move
    return best_move

        

def minimax(position: chess.Board, maximizing_player, depth: int = 5): # alpha: int = -1e9, beta: int = 1e9 
    """
    Minimax with alpha-beta pruning to save time
    """
    #just used minimax from connectx kaggle agent

    alpha = -1000 #could be -np.Inf
    beta = 1000
    if depth == 0:
        return nn_prediction(position)

    if maximizing_player:
        value = -np.Inf
        for move in position.legal_moves:
            position.push(move)
            value = max(value, minimax(position, False, depth - 1))
            position.pop()
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value

    else:
        value = np.Inf
        for move in position.legal_moves:
            position.push(move)
            value = min(value, minimax(position, True, depth - 1))
            position.pop()
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value

In [ ]:
#MIGHT NEED TO TAKE INTO ACCOUNT WHOSE TURN IN DATA...
find_best_move(chess.Board())